In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
movies_final_2016_df = pd.read_pickle('df_all_movies_2016_pk')
movies_final_2016_df.head(10)

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Finding Dory,"Jun 17, 2016",Walt Disney,Adventure,"$486,295,561","56,219,140",https://www.the-numbers.com/movie/Finding-Dory...,https://www.the-numbers.com/movie/Finding-Dory...
1,2,Rogue One: A Star Wars Story,"Dec 16, 2016",Walt Disney,Adventure,"$424,987,707","49,131,526",https://www.the-numbers.com/movie/Rogue-One-A-...,https://www.the-numbers.com/movie/Rogue-One-A-...
2,3,Captain America: Civil War,"May 6, 2016",Walt Disney,Action,"$408,084,349","47,177,381",https://www.the-numbers.com/movie/Captain-Amer...,https://www.the-numbers.com/movie/Captain-Amer...
3,4,The Secret Life of Pets,"Jul 8, 2016",Universal,Adventure,"$368,384,330","42,587,783",https://www.the-numbers.com/movie/Secret-Life-...,https://www.the-numbers.com/movie/Secret-Life-...
4,5,The Jungle Book,"Apr 15, 2016",Walt Disney,Adventure,"$364,001,123","42,081,054",https://www.the-numbers.com/movie/Jungle-Book-...,https://www.the-numbers.com/movie/Jungle-Book-...
5,6,Deadpool,"Feb 12, 2016",20th Century Fox,Action,"$363,070,709","41,973,492",https://www.the-numbers.com/movie/Deadpool#tab...,https://www.the-numbers.com/movie/Deadpool#tab...
6,7,Zootopia,"Mar 4, 2016",Walt Disney,Adventure,"$341,268,248","39,452,976",https://www.the-numbers.com/movie/Zootopia-(20...,https://www.the-numbers.com/movie/Zootopia-(20...
7,8,Batman v Superman: Dawn of …,"Mar 25, 2016",Warner Bros.,Action,"$330,360,194","38,191,930",https://www.the-numbers.com/movie/Batman-v-Sup...,https://www.the-numbers.com/movie/Batman-v-Sup...
8,9,Suicide Squad,"Aug 5, 2016",Warner Bros.,Action,"$325,100,054","37,583,821",https://www.the-numbers.com/movie/Suicide-Squa...,https://www.the-numbers.com/movie/Suicide-Squa...
9,10,Doctor Strange,"Nov 4, 2016",Walt Disney,Action,"$230,107,790","26,602,056",https://www.the-numbers.com/movie/Doctor-Stran...,https://www.the-numbers.com/movie/Doctor-Stran...


In [3]:
legs_2016_df = pd.read_pickle('legs_2016_df_pk')
legs_2016_df['Legs']

0       3.60
1       3.43
2       2.28
3       3.53
4       3.53
       ...  
801     1.67
802     1.00
803    12.92
804     3.15
805     1.23
Name: Legs, Length: 806, dtype: float64

In [4]:
movies_final_2016_df['Legs'] = legs_2016_df['Legs']

In [5]:
movies_final_2016_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs
0,1,Finding Dory,"Jun 17, 2016",Walt Disney,Adventure,"$486,295,561","56,219,140",https://www.the-numbers.com/movie/Finding-Dory...,https://www.the-numbers.com/movie/Finding-Dory...,3.60
1,2,Rogue One: A Star Wars Story,"Dec 16, 2016",Walt Disney,Adventure,"$424,987,707","49,131,526",https://www.the-numbers.com/movie/Rogue-One-A-...,https://www.the-numbers.com/movie/Rogue-One-A-...,3.43
2,3,Captain America: Civil War,"May 6, 2016",Walt Disney,Action,"$408,084,349","47,177,381",https://www.the-numbers.com/movie/Captain-Amer...,https://www.the-numbers.com/movie/Captain-Amer...,2.28
3,4,The Secret Life of Pets,"Jul 8, 2016",Universal,Adventure,"$368,384,330","42,587,783",https://www.the-numbers.com/movie/Secret-Life-...,https://www.the-numbers.com/movie/Secret-Life-...,3.53
4,5,The Jungle Book,"Apr 15, 2016",Walt Disney,Adventure,"$364,001,123","42,081,054",https://www.the-numbers.com/movie/Jungle-Book-...,https://www.the-numbers.com/movie/Jungle-Book-...,3.53
...,...,...,...,...,...,...,...,...,...,...
801,802,Green is Gold,"Oct 7, 2016",Samuel Goldwyn Films,Drama,$455,52,https://www.the-numbers.com/movie/Green-is-Gol...,https://www.the-numbers.com/movie/Green-is-Gol...,1.67
802,803,GÃ¼eros,"May 20, 2015",Kino Lorber,Comedy,$428,49,https://www.the-numbers.com/movie/Gueros#tab=s...,https://www.the-numbers.com/movie/Gueros#tab=c...,1.00
803,804,News From Planet Mars,"Jul 22, 2016",Kino Lorber,Comedy,$310,35,https://www.the-numbers.com/movie/Des-nouvelle...,https://www.the-numbers.com/movie/Des-nouvelle...,12.92
804,805,The Lady in the Car With Gl…,"Dec 18, 2015",Magnolia Pictures,Thriller/Suspense,$266,30,https://www.the-numbers.com/movie/dame-dans-l-...,https://www.the-numbers.com/movie/dame-dans-l-...,3.15


In [6]:
budget_2016_df = pd.read_pickle('budget_2016_pk')
wbo_to_budget_ratio_2016_df = pd.read_pickle('wbo_to_budget_ratio_2016_pk')

movies_final_2016_df['Budget'] = budget_2016_df['Production_Budget']
movies_final_2016_df['WBO_to_Budget_Ratio'] = (
    wbo_to_budget_ratio_2016_df['WBO to Budget Ratio'])

In [7]:
movies_final_2016_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio
0,1,Finding Dory,"Jun 17, 2016",Walt Disney,Adventure,"$486,295,561","56,219,140",https://www.the-numbers.com/movie/Finding-Dory...,https://www.the-numbers.com/movie/Finding-Dory...,3.60,200000000,5.1
1,2,Rogue One: A Star Wars Story,"Dec 16, 2016",Walt Disney,Adventure,"$424,987,707","49,131,526",https://www.the-numbers.com/movie/Rogue-One-A-...,https://www.the-numbers.com/movie/Rogue-One-A-...,3.43,200000000,5.3
2,3,Captain America: Civil War,"May 6, 2016",Walt Disney,Action,"$408,084,349","47,177,381",https://www.the-numbers.com/movie/Captain-Amer...,https://www.the-numbers.com/movie/Captain-Amer...,2.28,250000000,4.6
3,4,The Secret Life of Pets,"Jul 8, 2016",Universal,Adventure,"$368,384,330","42,587,783",https://www.the-numbers.com/movie/Secret-Life-...,https://www.the-numbers.com/movie/Secret-Life-...,3.53,75000000,11.8
4,5,The Jungle Book,"Apr 15, 2016",Walt Disney,Adventure,"$364,001,123","42,081,054",https://www.the-numbers.com/movie/Jungle-Book-...,https://www.the-numbers.com/movie/Jungle-Book-...,3.53,175000000,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...
801,802,Green is Gold,"Oct 7, 2016",Samuel Goldwyn Films,Drama,$455,52,https://www.the-numbers.com/movie/Green-is-Gol...,https://www.the-numbers.com/movie/Green-is-Gol...,1.67,nf,nf
802,803,GÃ¼eros,"May 20, 2015",Kino Lorber,Comedy,$428,49,https://www.the-numbers.com/movie/Gueros#tab=s...,https://www.the-numbers.com/movie/Gueros#tab=c...,1.00,nf,nf
803,804,News From Planet Mars,"Jul 22, 2016",Kino Lorber,Comedy,$310,35,https://www.the-numbers.com/movie/Des-nouvelle...,https://www.the-numbers.com/movie/Des-nouvelle...,12.92,nf,nf
804,805,The Lady in the Car With Gl…,"Dec 18, 2015",Magnolia Pictures,Thriller/Suspense,$266,30,https://www.the-numbers.com/movie/dame-dans-l-...,https://www.the-numbers.com/movie/dame-dans-l-...,3.15,nf,nf


In [8]:
theater_count_2016_df = pd.read_pickle('theater_count_2016_pk')
avg_run_2016_df = pd.read_pickle('avg_run_2016_pk')

movies_final_2016_df['Theater_Count'] = theater_count_2016_df['Theater_Count']
movies_final_2016_df['Average_Run'] = avg_run_2016_df['Average_Run']


In [9]:
source_2016_df = pd.read_pickle('source_2016_pk')
movies_final_2016_df['Source'] = source_2016_df['Source']
# movies_final_2016_df

In [10]:
creative_type_2016_df =pd.read_pickle('creative_type_2016_pk') 
movies_final_2016_df['Creative_Type'] = creative_type_2016_df['Creative_Type']
movies_final_2016_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Source,Creative_Type
0,1,Finding Dory,"Jun 17, 2016",Walt Disney,Adventure,"$486,295,561","56,219,140",https://www.the-numbers.com/movie/Finding-Dory...,https://www.the-numbers.com/movie/Finding-Dory...,3.60,200000000,5.1,4305,7.6,Original Screenplay,Kids Fiction
1,2,Rogue One: A Star Wars Story,"Dec 16, 2016",Walt Disney,Adventure,"$424,987,707","49,131,526",https://www.the-numbers.com/movie/Rogue-One-A-...,https://www.the-numbers.com/movie/Rogue-One-A-...,3.43,200000000,5.3,4157,7.0,Spin-Off,Science Fiction
2,3,Captain America: Civil War,"May 6, 2016",Walt Disney,Action,"$408,084,349","47,177,381",https://www.the-numbers.com/movie/Captain-Amer...,https://www.the-numbers.com/movie/Captain-Amer...,2.28,250000000,4.6,4226,6.0,Based on Comic/Graphic Novel,Super Hero
3,4,The Secret Life of Pets,"Jul 8, 2016",Universal,Adventure,"$368,384,330","42,587,783",https://www.the-numbers.com/movie/Secret-Life-...,https://www.the-numbers.com/movie/Secret-Life-...,3.53,75000000,11.8,4370,8.1,Original Screenplay,Kids Fiction
4,5,The Jungle Book,"Apr 15, 2016",Walt Disney,Adventure,"$364,001,123","42,081,054",https://www.the-numbers.com/movie/Jungle-Book-...,https://www.the-numbers.com/movie/Jungle-Book-...,3.53,175000000,5.4,4028,8.1,Based on Fiction Book/Short Story,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,802,Green is Gold,"Oct 7, 2016",Samuel Goldwyn Films,Drama,$455,52,https://www.the-numbers.com/movie/Green-is-Gol...,https://www.the-numbers.com/movie/Green-is-Gol...,1.67,nf,nf,1,1.0,Original Screenplay,Contemporary Fiction
802,803,GÃ¼eros,"May 20, 2015",Kino Lorber,Comedy,$428,49,https://www.the-numbers.com/movie/Gueros#tab=s...,https://www.the-numbers.com/movie/Gueros#tab=c...,1.00,nf,nf,nf,nf,Original Screenplay,Historical Fiction
803,804,News From Planet Mars,"Jul 22, 2016",Kino Lorber,Comedy,$310,35,https://www.the-numbers.com/movie/Des-nouvelle...,https://www.the-numbers.com/movie/Des-nouvelle...,12.92,nf,nf,1,1.0,Original Screenplay,Contemporary Fiction
804,805,The Lady in the Car With Gl…,"Dec 18, 2015",Magnolia Pictures,Thriller/Suspense,$266,30,https://www.the-numbers.com/movie/dame-dans-l-...,https://www.the-numbers.com/movie/dame-dans-l-...,3.15,nf,nf,2,3.5,Based on Fiction Book/Short Story,Historical Fiction


In [11]:
movies_final_2016_df.drop(['summary_page_url', 'cast_page_url'],
                          axis = 1, inplace = True)
movies_final_2016_df.shape

(806, 14)

In [12]:
col_reorder = ['Rank', 'Movie','ReleaseDate', 'Revenues', 'Tickets_Sold',
              'Legs', 'Budget', 'WBO_to_Budget_Ratio', 'Theater_Count',
               'Average_Run', 'Genre', 'Source', 'Creative_Type', 'Distributor']

movies_final_2016_df = movies_final_2016_df[col_reorder]

In [13]:
movies_final_2016_df

,Rank,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Genre,Source,Creative_Type,Distributor
0,1,Finding Dory,"Jun 17, 2016","$486,295,561","56,219,140",3.60,200000000,5.1,4305,7.6,Adventure,Original Screenplay,Kids Fiction,Walt Disney
1,2,Rogue One: A Star Wars Story,"Dec 16, 2016","$424,987,707","49,131,526",3.43,200000000,5.3,4157,7.0,Adventure,Spin-Off,Science Fiction,Walt Disney
2,3,Captain America: Civil War,"May 6, 2016","$408,084,349","47,177,381",2.28,250000000,4.6,4226,6.0,Action,Based on Comic/Graphic Novel,Super Hero,Walt Disney
3,4,The Secret Life of Pets,"Jul 8, 2016","$368,384,330","42,587,783",3.53,75000000,11.8,4370,8.1,Adventure,Original Screenplay,Kids Fiction,Universal
4,5,The Jungle Book,"Apr 15, 2016","$364,001,123","42,081,054",3.53,175000000,5.4,4028,8.1,Adventure,Based on Fiction Book/Short Story,Fantasy,Walt Disney
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,802,Green is Gold,"Oct 7, 2016",$455,52,1.67,nf,nf,1,1.0,Drama,Original Screenplay,Contemporary Fiction,Samuel Goldwyn Films
802,803,GÃ¼eros,"May 20, 2015",$428,49,1.00,nf,nf,nf,nf,Comedy,Original Screenplay,Historical Fiction,Kino Lorber
803,804,News From Planet Mars,"Jul 22, 2016",$310,35,12.92,nf,nf,1,1.0,Comedy,Original Screenplay,Contemporary Fiction,Kino Lorber
804,805,The Lady in the Car With Gl…,"Dec 18, 2015",$266,30,3.15,nf,nf,2,3.5,Thriller/Suspense,Based on Fiction Book/Short Story,Historical Fiction,Magnolia Pictures


In [14]:
cast_2016_df = pd.read_pickle('cast_2016_pk')
# cast_2016_df

In [15]:
movies_final_2016_df = movies_final_2016_df.join(cast_2016_df)

In [16]:
movies_final_2016_df.to_pickle('final_2016_pk')

In [17]:
nf_count_dict = {}

# name = 'Budget'
# movies_final_2016_df[movies_final_2016_df[name] == 'nf'][name].count()

for name in col_reorder:
    count_nf = movies_final_2016_df[movies_final_2016_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')


    
    

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   544
WBO_to_Budget_Ratio   544
Theater_Count   94
Average_Run   94
Genre   0
Source   1
Creative_Type   1
Distributor   0


In [18]:
movies_final_2016_df.dtypes
# movies_final_2016_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                object
Tickets_Sold            object
Legs                   float64
Budget                  object
WBO_to_Budget_Ratio     object
Theater_Count           object
Average_Run             object
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [19]:

movies_final_2016_df.replace(r'nf', np.nan, regex = True, inplace = True )


col_list = ['Revenues', 'Tickets_Sold', 'Legs', 'Budget', 'WBO_to_Budget_Ratio',
           'Theater_Count', 'Average_Run']

for col_name in col_list:
    movies_final_2016_df[col_name] = (movies_final_2016_df[col_name]
                                    .replace('[\$,]', '', regex = True))
                                        
    movies_final_2016_df[col_name] = (pd.to_numeric(movies_final_2016_df[col_name], 
                                                errors = 'coerce'))
    


In [20]:
movies_final_2016_df.dtypes
# movies_final_2016_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                 int64
Tickets_Sold             int64
Legs                   float64
Budget                 float64
WBO_to_Budget_Ratio    float64
Theater_Count          float64
Average_Run            float64
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [21]:
for name in col_reorder:
    count_nf = movies_final_2016_df[movies_final_2016_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   0
WBO_to_Budget_Ratio   0
Theater_Count   0
Average_Run   0
Genre   0
Source   0
Creative_Type   0
Distributor   0


In [22]:
for col_name in col_reorder:
    count_nan = movies_final_2016_df[col_name].isna().sum()
    print(f'{col_name}   {count_nan}')



Rank   0
Movie   5
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   544
WBO_to_Budget_Ratio   544
Theater_Count   94
Average_Run   94
Genre   0
Source   1
Creative_Type   1
Distributor   0


In [23]:
movies_final_2016_df.to_pickle('final_2016_pk')